# Terms of Use

### Please solve the authorization problem of the dataset on your own. You shall be solely responsible for any problems caused by the use of non-authorized datasets for training and all consequences thereof.The repository and its maintainer, svc develop team, have nothing to do with the consequences!

1. This project is established for academic exchange purposes only and is intended for communication and learning purposes. It is not intended for production environments.
2. Any videos based on sovits that are published on video platforms must clearly indicate in the description that they are used for voice changing and specify the input source of the voice or audio, for example, using videos or audios published by others and separating the vocals as input source for conversion, which must provide clear original video or music links. If your own voice or other synthesized voices from other commercial vocal synthesis software are used as the input source for conversion, you must also explain it in the description.
3. You shall be solely responsible for any infringement problems caused by the input source. When using other commercial vocal synthesis software as input source, please ensure that you comply with the terms of use of the software. Note that many vocal synthesis engines clearly state in their terms of use that they cannot be used for input source conversion.
4. Continuing to use this project is deemed as agreeing to the relevant provisions stated in this repository README. This repository README has the obligation to persuade, and is not responsible for any subsequent problems that may arise.
5. If you distribute this repository's code or publish any results produced by this project publicly (including but not limited to video sharing platforms), please indicate the original author and code source (this repository).
6. If you use this project for any other plan, please contact and inform the author of this repository in advance. Thank you very much.


## **Note:**
## **Make sure there is no a directory named `sovits4data` in your google drive at the first time you use this notebook.**
## **It will be created to store some necessary files.**
## **For sure you can change it to another directory by modifying `sovits_data_dir` variable.**

# **Initialize environment**

In [1]:
#@title Connect to colab runtime and check GPU

#@markdown # Connect to colab runtime and check GPU

#@markdown

!nvidia-smi

Fri Mar 28 08:13:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
#@title Clone repository and install requirements

#@markdown # Clone repository and install requirements

#@markdown

#@markdown ### After the execution is completed, the runtime will **automatically restart**

#@markdown

!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.1-Stable
%cd /content/so-vits-svc
%pip install --upgrade pip setuptools
%pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118
exit()

fatal: destination path 'so-vits-svc' already exists and is not an empty directory.
/content/so-vits-svc
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached gradio-5.23.1-py3-none-any.whl.metadata (16 kB)
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pyworld-0.3.5.tar.gz (261 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 81.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.3 

In [1]:
#@title Mount google drive and select which directories to sync with google drive

#@markdown # Mount google drive and select which directories to sync with google drive

#@markdown

from google.colab import drive
drive.mount("/content/drive")

#@markdown Directory to store **necessary files**, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  #@param {type:"string"}
#@markdown By default it will create a `sovits4data/` folder in your google drive.
RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"

#@markdown

#@markdown ### These folders will be synced with your google drvie

#@markdown　### **Strongly recommend to check all.**

#@markdown Sync **input audios** and **output audios**
sync_raw_and_results = True  #@param {type:"boolean"}
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results

#@markdown Sync **config** and **models**
sync_configs_and_logs = True  #@param {type:"boolean"}
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k

Mounted at /content/drive


In [2]:
#@title Get pretrained model(Optional but strongly recommend).

#@markdown # Get pretrained model(Optional but strongly recommend).

#@markdown

#@markdown - Pre-trained model files: `G_0.pth` `D_0.pth`
#@markdown   - Place them under /sovits4data/logs/44k/ in your google drive manualy

#@markdown Get them from svc-develop-team(TBD) or anywhere else.

#@markdown Although the pretrained model generally does not cause any copyright problems, please pay attention to it. For example, ask the author in advance, or the author has indicated the feasible use in the description clearly.

download_pretrained_model = True #@param {type:"boolean"}
D_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_D_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_D_320000.pth"] {allow-input: true}
G_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_G_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_G_320000.pth"] {allow-input: true}

download_pretrained_diffusion_model = True #@param {type:"boolean"}
diff_model_URL = "https://huggingface.co/datasets/ms903/Diff-SVC-refactor-pre-trained-model/resolve/main/fix_pitch_add_vctk_600k/model_0.pt" #@param {type:"string"}

%cd /content/so-vits-svc

if download_pretrained_model:
    !curl -L {D_0_URL} -o logs/44k/D_0.pth
    !md5sum logs/44k/D_0.pth
    !curl -L {G_0_URL} -o logs/44k/G_0.pth
    !md5sum logs/44k/G_0.pth

if download_pretrained_diffusion_model:
    !mkdir -p logs/44k/diffusion
    !curl -L {diff_model_URL} -o logs/44k/diffusion/model_0.pt
    !md5sum logs/44k/diffusion/model_0.pt

/content/so-vits-svc
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1107  100  1107    0     0   7057      0 --:--:-- --:--:-- --:--:--  7096
100  178M  100  178M    0     0  66.8M      0  0:00:02  0:00:02 --:--:-- 73.9M
791dfe20d626a7eca67c5c10420d327f  logs/44k/D_0.pth
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1107  100  1107    0     0   6276      0 --:--:-- --:--:-- --:--:--  6289
100  199M  100  199M    0     0  73.1M      0  0:00:02  0:00:02 --:--:-- 83.2M
8249cd8bbfd6a69ffad5125ad169c70f  logs/44k/G_0.pth
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1101  100  1101    0     0   8159      0 --:--:-- --:--:-- --:--:--  8216
100  210

In [7]:
!cp /content/drive/MyDrive/siratori-idol/* /content/drive/MyDrive/sovits4data/merged_speaker_data/siratori-idol

In [8]:

# Create zip file
with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(target_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, target_dir))

print(f"Zipped files saved at: {zip_filepath}")


Zipped files saved at: /content/drive/MyDrive/sovits4data/speaker_data.zip


# **Dataset preprocessing**

Pack and upload your raw dataset(dataset_raw/) to your google drive.

Makesure the file structure in your zip file looks like this:

```
YourZIPforSingleSpeakers.zip
└───speaker
    ├───xxx1-xxx1.wav
    ├───...
    └───Lxx-0xx8.wav
```

```
YourZIPforMultipleSpeakers.zip
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav
```

**Even if there is only one speaker, a folder named `{speaker_name}` is needed.**

In [9]:
#@title Get raw dataset from google drive

#@markdown # Get raw dataset from google drive

#@markdown

#@markdown Directory where **your zip file** located in, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  #@param {type:"string"}
#@markdown Filename of **your zip file**, do NOT be "dataset.zip"
zip_filename = "speaker_data.zip"  #@param {type:"string"}
ZIP_PATH = sovits_data_dir + zip_filename

!unzip -od /content/so-vits-svc/dataset_raw {ZIP_PATH}

Archive:  /content/drive/MyDrive/sovits4data/speaker_data.zip
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-114.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-115.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-116.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-117.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-118.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-119.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-120.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-121.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-122.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-123.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-124.wav  
  inflating: /content/so-vits-svc/dataset_raw/saionji-idol/saionji-125.wav  
  inflating: /

In [ ]:
# # prompt: lets create a zip file insude sovits_data_dir. and inside the zip we will have saionji-idol, saotome-idol_, siratori-idol folders in it

# import os
# import zipfile

# # Define paths
# sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
# zip_filepath = os.path.join(sovits_data_dir, "speaker_data.zip")  # Name your zip file
# saionji_idol_dir = "/content/so-vits-svc/dataset_raw/saionji-idol"
# saotome_idol_dir = "/content/so-vits-svc/dataset_raw/saotome-idol_"
# siratori_idol_dir = "/content/so-vits-svc/dataset_raw/siratori-idol"

# # Create dummy directories and files (replace with your actual data)
# os.makedirs(saionji_idol_dir, exist_ok=True)
# os.makedirs(saotome_idol_dir, exist_ok=True)
# os.makedirs(siratori_idol_dir, exist_ok=True)

# with open(os.path.join(saionji_idol_dir, "dummy.txt"), "w") as f:
#     f.write("Saionji Idol")
# with open(os.path.join(saotome_idol_dir, "dummy.txt"), "w") as f:
#     f.write("Saotome Idol")
# with open(os.path.join(siratori_idol_dir, "dummy.txt"), "w") as f:
#     f.write("Siratori Idol")


# # Create the zip file
# with zipfile.ZipFile(zip_filepath, "w", zipfile.ZIP_DEFLATED) as zipf:
#     for root, _, files in os.walk("/content/so-vits-svc/dataset_raw"):
#         for file in files:
#             file_path = os.path.join(root, file)
#             arcname = os.path.relpath(file_path, "/content/so-vits-svc/dataset_raw")
#             zipf.write(file_path, arcname=arcname)

# print(f"Zip file created at: {zip_filepath}")


In [10]:
#@title Resample to 44100Hz and mono

#@markdown # Resample to 44100Hz and mono

#@markdown

%cd /content/so-vits-svc
!python resample.py

/content/so-vits-svc
CPU count: 2
./dataset_raw/siratori-idol
resampling: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:19
./dataset_raw/saotome-idol
resampling: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
./dataset_raw/saionji-idol
resampling: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00


In [12]:
# prompt: Traceback (most recent call last):
#   File "/content/so-vits-svc/preprocess_flist_config.py", line 8, in <module>
#     from loguru import logger
# ModuleNotFoundError: No module named 'loguru'

!pip install loguru


  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)


In [13]:
#@title Divide filelists and generate config.json

#@markdown # Divide filelists and generate config.json

#@markdown

%cd /content/so-vits-svc

speech_encoder = "whisper-ppg-large" #@param ["vec768l12", "vec256l9", "hubertsoft", "whisper-ppg", "whisper-ppg-large"]
use_vol_aug = True #@param {type:"boolean"}
vol_aug = "--vol_aug" if use_vol_aug else ""

from pretrain.meta import download_dict
download_dict = download_dict()

url = download_dict[speech_encoder]["url"]
output = download_dict[speech_encoder]["output"]

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}
  !md5sum {output}

!python preprocess_flist_config.py --speech_encoder={speech_encoder} {vol_aug}

/content/so-vits-svc
  0% 0/3 [00:00<?, ?it/s]2025-03-28 08:41:34.149 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-128.wav
2025-03-28 08:41:34.155 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-125.wav
2025-03-28 08:41:34.156 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-22.wav
2025-03-28 08:41:34.159 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-77.wav
2025-03-28 08:41:34.160 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-60.wav
2025-03-28 08:41:34.161 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-126.wav
2025-03-28 08:41:34.163 | WARNING  | __main__:<module>:59 - Detected non-ASCII file name: ./dataset/44k/siratori-idol/siratori-164.wav
2025-03-28 0

In [18]:
# prompt: renamed 'nsf_hifigan' -> 'pretrain/nsf_hifigan'
# Traceback (most recent call last):
#   File "/content/so-vits-svc/preprocess_hubert_f0.py", line 17, in <module>
#     import utils
#   File "/content/so-vits-svc/utils.py", line 12, in <module>
#     import faiss
# ModuleNotFoundError: No module named 'faiss'

!pip install faiss-cpu



  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 119.5 MB/s eta 0:00:00


In [20]:
!pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6128 sha256=b18f0a70b72a4a3356d86b3443df28b7d203d32ea7394f472cef618aa8f43d32
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg


In [26]:
!pip install pip==23.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [27]:
!pip install -r /content/so-vits-svc/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 24.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.

In [1]:
#@title Generate hubert and f0

#@markdown # Generate hubert and f0

#@markdown
%cd /content/so-vits-svc

f0_predictor = "rmvpe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]
use_diff = True #@param {type:"boolean"}

import os
if f0_predictor == "rmvpe" and not os.path.exists("./pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("./pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt


diff_param = ""
if use_diff:
  diff_param = "--use_diff"

  if not os.path.exists("./pretrain/nsf_hifigan/model"):
    !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
    !md5sum nsf_hifigan_20221211.zip
    !unzip nsf_hifigan_20221211.zip
    !rm -rf pretrain/nsf_hifigan
    !mv -v nsf_hifigan pretrain

!python preprocess_hubert_f0.py --f0_predictor={f0_predictor} {diff_param}

/content/so-vits-svc
whisper-ppg-large
2025-03-28 08:58:44.620 | INFO     | __main__:<module>:152 - Using device: cuda:0
2025-03-28 08:58:44.620 | INFO     | __main__:<module>:153 - Using SpeechEncoder: whisper-ppg-large
2025-03-28 08:58:44.620 | INFO     | __main__:<module>:154 - Using extractor: rmvpe
2025-03-28 08:58:44.620 | INFO     | __main__:<module>:155 - Using diff Mode: True
use_diff
Loading Mel Extractor...
Loaded Mel Extractor.
  0% 0/1 [00:00<?, ?it/s]2025-03-28 08:58:49.600 | INFO     | __mp_main__:process_batch:107 - Loading speech encoder for content...
2025-03-28 08:58:49.626 | INFO     | __mp_main__:process_batch:113 - Rank 1 uses device cuda:0
2025-03-28 08:59:26.611 | INFO     | __mp_main__:process_batch:115 - Loaded speech encoder for rank 1

  0% 0/633 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all input

In [2]:
#@title Save the preprocessed dataset to google drive

#@markdown # Save the preprocessed dataset to google drive

#@markdown

#@markdown You can save the dataset and related files to your google drive for the next training

#@markdown **Directory for saving**, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/" #@param {type:"string"}

#@markdown There will be a `dataset.zip` contained `dataset/` in your google drive, which is preprocessed data.

!mkdir -p {sovits_data_dir}
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp -vr dataset.zip "{sovits_data_dir}"

Streaming output truncated to the last 5000 lines.
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-87.wav.aug_mel.npy (deflated 15%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-156.spec.pt (deflated 15%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-185.wav.aug_vol.npy (deflated 9%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-164.wav (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-207.wav.aug_mel.npy (deflated 24%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-185.wav.vol.npy (deflated 9%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-57.wav.aug_vol.npy (deflated 7%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-127.wav.f0.npy (deflated 69%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-87.spec.pt (deflated 10%)
  adding: content/so-vits-svc/dataset/44k/siratori-idol/siratori-206.wav.aug_vol.npy (deflate

In [3]:
#@title Unzip preprocessed dataset from google drive directly if you have preprocessed already.

#@markdown # Unzip preprocessed dataset from google drive directly if you have preprocessed already.

#@markdown

#@markdown Directory where **your preprocessed dataset** located in, dont miss the slash at the end👇.
sovits_data_dir = "/content/drive/MyDrive/sovits4data/" #@param {type:"string"}
CONFIG = sovits_data_dir + "configs/"
FILELISTS = sovits_data_dir + "filelists/"
DATASET = sovits_data_dir + "dataset.zip"

!cp -vr {CONFIG} /content/so-vits-svc/
!cp -vr {FILELISTS} /content/so-vits-svc/
!unzip {DATASET} -d /

Streaming output truncated to the last 5000 lines.
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-122.spec.pt  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-71.wav.mel.npy  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-113.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-87.wav.aug_mel.npy  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-156.spec.pt  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-185.wav.aug_vol.npy  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-164.wav  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-207.wav.aug_mel.npy  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-185.wav.vol.npy  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-57.wav.aug_vol.npy  
  inflating: /content/so-vits-svc/dataset/44k/siratori-idol/siratori-127.wav.f0.np

# **Trainning**

In [2]:
!pip uninstall numpy -y



Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement Flask_Cors==3.1.1 (from versions: 0.0.0.dev3, 0.0.0.dev4, 1.0, 1.1, 1.1.1, 1.1.2, 1.1.3, 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.6.0, 1.6.1, 1.7.0, 1.7.1, 1.7.2, 1.7.3, 1.7.4, 1.8.0, 1.8.1, 1.9.0, 1.10.0, 1.10.1, 1.10.2, 1.10.3, 2.0.0rc1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.0.4, 3.0.6, 3.0.7, 3.0.8, 3.0.9, 3.0.10, 4.0.0a0, 4.0.0, 4.0.1, 4.0.2, 5.0.0, 5.0.1)
ERROR: No matching distribution found for Flask_Cors==3.1.1


In [4]:
#@title Start training

#@markdown # Start training

#@markdown If you want to use pre-trained models, upload them to /sovits4data/logs/44k/ in your google drive manualy.

#@markdown

%cd /content/so-vits-svc

#@markdown Whether to enable tensorboard
# tensorboard_on = True  #@param {type:"boolean"}

# if tensorboard_on:
#   %load_ext tensorboard
#   %tensorboard --logdir logs/44k

config_path = "configs/config.json"

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}

!python train.py -c {config_path} -m 44k

/content/so-vits-svc
2025-03-28 10:47:31.459982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743158851.483091   41422 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743158851.490007   41422 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 10:47:31.514470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and

In [3]:
pip install --upgrade numpy


  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.2
    Uninstalling numpy-1.23.2:
      Successfully uninstalled numpy-1.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.8.0.post1 requires numpy<2.0,>=1.0, but you have numpy 2.2.4 which is incompatible.
scipy 1.10.0 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.0 which is incompatible.
imbalanced-learn 0.13.0 requires sc

In [ ]:
#@title Train cluster model (Optional)

#@markdown # Train cluster model (Optional)

#@markdown #### Details see [README.md#cluster-based-timbre-leakage-control](https://github.com/svc-develop-team/so-vits-svc#cluster-based-timbre-leakage-control)

#@markdown

%cd /content/so-vits-svc
!python cluster/train_cluster.py --gpu

In [ ]:
#@title Train index model (Optional)

#@markdown # Train index model (Optional)

#@markdown #### Details see [README.md#feature-retrieval](https://github.com/svc-develop-team/so-vits-svc#feature-retrieval)

#@markdown

%cd /content/so-vits-svc
!python train_index.py -c configs/config.json

In [ ]:
#@title Train diffusion model (Optional)

#@markdown # Train diffusion model (Optional)

#@markdown #### Details see [README.md#-about-shallow-diffusion](https://github.com/svc-develop-team/so-vits-svc#-about-shallow-diffusion)

#@markdown

%cd /content/so-vits-svc

import os
if not os.path.exists("./pretrain/nsf_hifigan/model"):
  !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
  !unzip nsf_hifigan_20221211.zip
  !rm -rf pretrain/nsf_hifigan
  !mv -v nsf_hifigan pretrain

#@markdown Whether to enable tensorboard
tensorboard_on = True  #@param {type:"boolean"}

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train_diff.py -c configs/diffusion.yaml

# keep colab alive
Open the devtools and copy & paste to run the scrips.


```JavaScript
const ping = () => {
  const btn = document.querySelector("colab-connect-button");
  const inner_btn = btn.shadowRoot.querySelector("#connect");
  if (inner_btn) {
    inner_btn.click();
    console.log("Clicked on connect button");
  } else {
    console.log("connect button not found");
  }

  const nextTime = 50000 + Math.random() * 10000;

  setTimeout(ping, nextTime);
};

ping();
```

# **Inference**
### Upload wav files from this notebook
### **OR**
### Upload to `sovits4data/raw/` in your google drive manualy (should be faster)

In [ ]:
#title Download nsf_hifigan if you need it

%cd /content/so-vits-svc
!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /content/so-vits-svc/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain


In [ ]:
#@title Upload wav files, the filename should not contain any special symbols like `#` `$` `(` `)`

#@markdown # Upload wav files, the filename should not contain any special symbols like `#` `$` `(` `)`

#@markdown

%cd /content/so-vits-svc
%run wav_upload.py --type audio

In [ ]:
#@title Start inference (and download)

#@markdown # Start inference (and download)

#@markdown Parameters see [README.MD#Inference](https://github.com/svc-develop-team/so-vits-svc#-inference)

#@markdown

wav_filename = "YourWAVFile.wav"  #@param {type:"string"}
model_filename = "G_210000.pth"  #@param {type:"string"}
model_path = "/content/so-vits-svc/logs/44k/" + model_filename
speaker = "YourSpeaker"  #@param {type:"string"}
trans = "0"  #@param {type:"string"}
cluster_infer_ratio = "0"  #@param {type:"string"}
auto_predict_f0 = False  #@param {type:"boolean"}
apf = ""
if auto_predict_f0:
  apf = " -a "

f0_predictor = "crepe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]

enhance = False  #@param {type:"boolean"}
ehc = ""
if enhance:
  ehc = " -eh "
#@markdown

#@markdown Generally keep default:
config_filename = "config.json"  #@param {type:"string"}
config_path = "/content/so-vits-svc/configs/" + config_filename

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os

if f0_predictor == "rmvpe" and not os.path.exists("./pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("./pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt

if not os.path.exists(output):
  !curl -L {url} -o {output}

kmeans_filenname = "kmeans_10000.pt"  #@param {type:"string"}
kmeans_path = "/content/so-vits-svc/logs/44k/" + kmeans_filenname
slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}

key = "auto" if auto_predict_f0 else f"{trans}key"
cluster_name = "" if cluster_infer_ratio == "0" else f"_{cluster_infer_ratio}"
isdiffusion = "sovits"
wav_output = f"/content/so-vits-svc/results/{wav_filename}_{key}_{speaker}{cluster_name}_{isdiffusion}_{f0_predictor}.{wav_format}"

%cd /content/so-vits-svc
!python inference_main.py -n {wav_filename} -m {model_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -c {config_path} -cm {kmeans_path} -sd {slice_db} -wf {wav_format} {apf} --f0_predictor={f0_predictor} {ehc}

#@markdown

#@markdown If you dont want to download from here, uncheck this.
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)